# Code Manekineko Operation Practice

## Preperation work

In [1]:
import re
import tushare as ts
import pandas as pd
from datetime import *
from pandas import DataFrame, Series
import json

In [2]:
# Initialize the databse connection
from pymongo import MongoClient
# uri = 'mongodb://localhost'
uri='mongodb://code-manekineko:woL44kcE6OD3yUbrFuZ7Rv06wsDGRpblJYU1yWek8GDogbhoubho5mhihCgBDDnKAhluofqykKecFN1hAwUQXg==@code-manekineko.documents.azure.com:10255/?ssl=true&replicaSet=globaldb'

client = MongoClient(uri)
db = client['CodeManekineko']

In [3]:
c_stock_list = db['StockList']
c_article_body = db['ArticleBody']

To review the StockList count

In [9]:
df_basic_data = ts.get_stock_basics()

In [22]:
def get_stock_timeToMarket(stock_id):
    """
    Get timeToMarket by given stock id
    :param stock_id: 
    :return: Str object
    """    
    time_to_market=''
    
    for i in df_basic_data.index:
        if i == stock_id:
            time_to_market = df_basic_data.timeToMarket[i]

    if time_to_market == '':
        raise Exception('No timeToMarket value found for %s' % stock_id)

    return str(time_to_market)

# Information Gathering

## Stock metadata gathering from Tushare

Objective: To gather stock metadata from Tushare API

Target collection: StockList

Exception control
* Make sure the stock not exists in StockList

### StockList collection structure

In [4]:
c_stock_list.find()[0]

{'_id': ObjectId('58b054a0ed2c88418c9b908d'),
 'related_articles': [{'article_date': '2017-02-24',
   'article_id': ObjectId('58b0549fed2c88418c9b908c'),
   'article_name': 'A股震荡上行：任泽平认为结构性牛市或来临 重点关注蓝筹股',
   'url': 'http://finance.sina.com.cn/stock/marketresearch/2017-02-24/doc-ifyavvsh6483169.shtml'},
  {'article_date': '2017-02-24',
   'article_id': ObjectId('58b054d9ed2c88418c9b9211'),
   'article_name': '海外映射难再“拉郎配” 解码A股共振点',
   'url': 'http://finance.sina.com.cn/stock/marketresearch/2017-02-24/doc-ifyavwcv8660345.shtml'},
  {'article_date': '2017-02-22',
   'article_id': ObjectId('58b0555eed2c88418c9b936b'),
   'article_name': '三成个股已然突破3300点价位 A股开启局部牛市？',
   'url': 'http://finance.sina.com.cn/stock/marketresearch/2017-02-22/doc-ifyavvsh5956157.shtml'},
  {'article_date': '2017-02-22',
   'article_id': ObjectId('58b05587ed2c88418c9b93e4'),
   'article_name': '任泽平：四大超预期支持战略看多2017年A股市场',
   'url': 'http://finance.sina.com.cn/stock/marketresearch/2017-02-22/doc-ifyarrcf5353441.shtml'}

### Stock basic business information

Tushare API returns below data structure

#### 基本信息
>```py
import tushare as ts
ts.get_stock_basics()
```

* code,代码
* name,名称
* industry,所属行业
* area,地区
* pe,市盈率
* outstanding,流通股本(亿)
* totals,总股本(亿)
* totalAssets,总资产(万)
* liquidAssets,流动资产
* fixedAssets,固定资产
* reserved,公积金
* reservedPerShare,每股公积金
* esp,每股收益
* bvps,每股净资
* pb,市净率
* timeToMarket,上市日期
* undp,未分利润
* perundp, 每股未分配
* rev,收入同比(%)
* profit,利润同比(%)
* gpr,毛利率(%)
* npr,净利润率(%)
* holders,股东人数

#### 业绩报告（主表）
>```py
#获取2014年第3季度的业绩报表数据
ts.get_report_data(2014,3)
```
* code,代码
* name,名称
* esp,每股收益
* eps_yoy,每股收益同比(%)
* bvps,每股净资产
* roe,净资产收益率(%)
* epcf,每股现金流量(元)
* net_profits,净利润(万元)
* profits_yoy,净利润同比(%)
* distrib,分配方案
* report_date,发布日期

#### 盈利能力
>```py
#获取2014年第3季度的盈利能力数据
ts.get_profit_data(2014,3)
```
* code,代码
* name,名称
* roe,净资产收益率(%)
* net_profit_ratio,净利率(%)
* gross_profit_rate,毛利率(%)
* net_profits,净利润(万元)
* esp,每股收益
* business_income,营业收入(百万元)
* bips,每股主营业务收入(元)

#### 运营能力
>```py
#获取2014年第3季度的营运能力数据
ts.get_operation_data(2014,3)
```
* code,代码
* name,名称
* arturnover,应收账款周转率(次)
* arturndays,应收账款周转天数(天)
* inventory_turnover,存货周转率(次)
* inventory_days,存货周转天数(天)
* currentasset_turnover,流动资产周转率(次)
* currentasset_days,流动资产周转天数(天)

#### 成长能力
>```py
#获取2014年第3季度的成长能力数据
ts.get_growth_data(2014,3)
```

* code,代码
* name,名称
* mbrg,主营业务收入增长率(%)
* nprg,净利润增长率(%)
* nav,净资产增长率
* targ,总资产增长率
* epsg,每股收益增长率
* seg,股东权益增长率

#### 偿债能力
>```py
#获取2014年第3季度的偿债能力数据
ts.get_debtpaying_data(2014,3)
```
* code,代码
* name,名称
* currentratio,流动比率
* quickratio,速动比率
* cashratio,现金比率
* icratio,利息支付倍数
* sheqratio,股东权益比率
* adratio,股东权益增长率

#### 现金流量
>```py
#获取2014年第3季度的现金流量数据
ts.get_cashflow_data(2014,3)
```
* code,代码
* name,名称
* cf_sales,经营现金净流量对销售收入比率
* rateofreturn,资产的经营现金流量回报率
* cf_nm,经营现金净流量与净利润的比率
* cf_liabilities,经营现金净流量对负债比率
* cashflowratio,现金流量比率

In [6]:
# 获取股票基本信息
# 所返回的类型是pandas.core.frame.DataFrame
dfBasic = ts.get_stock_basics()
dfBasic.count()

name                3277
industry            3277
area                3277
pe                  3277
outstanding         3277
totals              3277
totalAssets         3277
liquidAssets        3277
fixedAssets         3277
reserved            3277
reservedPerShare    3277
esp                 3277
bvps                3277
pb                  3277
timeToMarket        3277
undp                3277
perundp             3277
rev                 3277
profit              3277
gpr                 3277
npr                 3277
holders             3277
dtype: int64

In [42]:
# Save stock basic info to csv file
# dfBasic.to_excel('stock_basic.xlsx', sheet_name='Sheet1')
dfBasic.timeToMarket['603586']

20170406

In [188]:
#Update existing stock information for downloaded basic info.
for i in dfBasic.values:
    doc = c_stock_list.find_one({'stock_name':i[0]})
    if doc is not None:
        doc['industry']=i[1]
        doc['area']=i[2]
        doc['pe']=i[3]
        doc['outstanding']=i[4]
        doc['totals']=i[5]
        doc['totalAssets']=i[6]
        doc['liquidAssets']=i[7]
        doc['fixedAssets']=i[8]
        doc['reserved']=i[9]
        doc['reservedPerShare']=i[10]
        doc['esp']=i[11]
        doc['bvps']=i[12]
        doc['pb']=i[13]
        doc['timeToMarket']=i[14]
        doc['undp']=i[15]
        doc['perundp']=i[16]
        doc['rev']=i[17]
        doc['profit']=i[18]
        doc['gpr']=i[19]
        doc['npr']=i[20]
        doc['holders']=i[21]
        
        c_stock_list.replace_one({'stock_name':i[0]},doc)


In [9]:
#更新业绩报告
dfReports = ts.get_report_data(2017,1)

[Getting data:]######################################################

In [10]:
dfReports[1:10]

,code,name,eps,eps_yoy,bvps,roe,epcf,net_profits,profits_yoy,distrib,report_date
1,603197,保隆科技,0.51,30.62,NaN,NaN,NaN,4518.35,30.61,NaN,05-18
2,603269,海鸥股份,0.08,0.00,6.23,1.33,0.12,578.00,0.18,NaN,05-16
3,603758,秦安股份,0.16,-15.79,NaN,3.60,NaN,6069.41,-13.67,NaN,05-16
4,300653,正海生物,0.13,0.93,3.95,3.35,0.14,777.96,0.89,NaN,05-15
5,300652,雷迪克,0.25,56.25,4.39,5.86,0.15,1649.39,58.76,NaN,05-15
6,002870,香山股份,0.25,38.89,NaN,5.69,NaN,2034.61,33.08,NaN,05-12
7,002869,金溢科技,0.48,2.13,5.02,9.51,-1.37,4261.38,2.04,NaN,05-12
8,601952,苏垦农发,0.17,21.43,3.37,5.23,0.20,13725.47,22.27,NaN,05-12
9,300660,江苏雷利,NaN,NaN,NaN,NaN,NaN,6067.51,21.42,NaN,05-11


In [11]:
# Utility 1. Check for existing list items in document by stock id, year, quarter
def ListItemExists(stock_name,year,quarter,list_name):
    doc=c_stock_list.find_one({'stock_name':stock_name})
    count=0
    
    if list_name in doc:
        for l in doc[list_name]:     
            if l['year']==year and l['quarter']==quarter:
                count += 1

    if count>1:
        print('Found %s duplicate %s in %s(%s-%s)' %(count, list_name, stock_name, year, quarter))
        
    if count>0:
        return True
    else:
        return False

In [12]:
# Utility 2. Build performance report data
def build_report_data(lst):
    data={
        'esp':lst[2],
        'eps_yoy':lst[3],
        'bvps':lst[4],
        'roe':lst[5],
        'epcf':lst[6],
        'net_profits':lst[7],
        'profits_yoy':lst[8],
        'distrib':lst[9],
        'report_date':lst[10]
    }
    return data

def build_profit_data(lst):
    data = {
        'roe':lst[2],
        'net_profile_ratio':lst[3],
        'gross_profit_rate':lst[4],
        'net_profits':lst[5],
        'esp':lst[6],
        'business_income':lst[7],
        'bips':lst[8]
    }    
    return data

def build_operation_data(lst):
    data = {
        'arturnover':lst[2],
        'arturndays':lst[3],
        'inventory_turnover':lst[4],
        'inventory_days':lst[5],
        'currentasset_turnover':lst[6],
        'currentasset_days':lst[7]
    }   
    return data

def build_growth_data(lst):
    data = {
        'mbrg':lst[2],
        'nprg':lst[3],
        'nav':lst[4],
        'targ':lst[5],
        'epsg':lst[6],
        'seg':lst[7]
    }   
    return data

def build_debtpaying_data(lst):
    data = {
        'currentratio':lst[2],
        'quickratio':lst[3],
        'cashratio':lst[4],
        'icratio':lst[5],
        'sheqratio':lst[6],
        'adratio':lst[7]
    }   
    return data

def build_cashflow_data(lst):
    data = {
        'cf_sales':lst[2],
        'rateofreturn':lst[3],
        'cf_nm':lst[4],
        'cf_liabilities':lst[5],
        'cashflowratio':lst[6]
    }   
    return data

In [13]:
def UpdateList(df, year, quarter, lst_name):
    if df is not None:
        success=0
        ignore=0
        
        for i in df.values:
            doc = c_stock_list.find_one({'stock_name':i[1]})
            if doc is not None:
                # Build report line items
                if lst_name not in doc:
                    doc[lst_name]=[]
                
                item=None
                
                if not ListItemExists(i[1], year, quarter,lst_name):
                    # Build item data
                    if lst_name == 'report_data':
                        item=build_report_data(i)
                    elif lst_name=='profit_data':
                        item=build_profit_data(i)
                    elif lst_name=='operation_data':
                        item=build_operation_data(i)
                    elif lst_name=='growth_data':
                        item=build_growth_data(i)
                    elif lst_name=='debtpaying_data':
                        item=build_debtpaying_data(i)
                    elif lst_name=='cashflow_data':
                        item=build_cashflow_data(i)
                    else:
                        print('Unknown list name, process abort.')
                        
                    item['year']=year
                    item['quarter']=quarter

                    doc[lst_name].append(item)

                    c_stock_list.replace_one({'stock_name':i[1]}, doc)
                    success+=1
                else:
                    ignore+=1
        print('%s(%s-%s) total:%s, success:%s, ignore:%s' %(lst_name,
                                                           year,
                                                           quarter,
                                                           success+ignore,
                                                           success,
                                                           ignore))
                    
            

In [24]:
for yr in range(datetime.now().year,1989,-1):
    for qr in range(1,4):
        try:
            report=ts.get_report_data(yr,qr)
            UpdateList(report, yr, qr, 'report_data')
        except Exception as e:
            print('Fail during getting report data on %s-%s, exception:%s' %(yr,qr,e))
        
        try:
            profit=ts.get_profit_data(yr,qr)
            UpdateList(profit, yr, qr, 'profit_data')
        except Exception as e:
            print('Fail during getting profit data on %s-%s, exception:%s' %(yr,qr,e))
        
        try:
            operation=ts.get_operation_data(yr,qr)
            UpdateList(operation, yr, qr, 'operation_data')
        except Exception as e:
            print('Fail during getting operation data on %s-%s, exception:%s' %(yr,qr,e))
        
        try:
            growth=ts.get_growth_data(yr,qr)
            UpdateList(growth, yr, qr, 'growth_data')
        except Exception as e:
            print('Fail during getting growth data on %s-%s, exception:%s' %(yr,qr,e))
        
        try:
            debtpaying=ts.get_debtpaying_data(yr,qr)
            UpdateList(debtpaying, yr, qr, 'debtpaying_data')
        except Exception as e:
            print('Fail during getting debtpaying data on %s-%s, exception:%s' %(yr,qr,e))
        
        try:
            cashflow=ts.get_cashflow_data(yr,qr)
            UpdateList(cashflow, yr, qr, 'cashflow_data')
        except Exception as e:
            print('Fail during getting cashflow data on %s-%s, exception:%s' %(yr,qr,e))
        
        

[Getting data:]######################################################report_data(2017-1) total:696, success:144, ignore:552
[Getting data:]######################################################profit_data(2017-1) total:696, success:3, ignore:693
[Getting data:]######################################################operation_data(2017-1) total:696, success:21, ignore:675
[Getting data:]######################################################growth_data(2017-1) total:683, success:17, ignore:666
[Getting data:]######################################################debtpaying_data(2017-1) total:696, success:0, ignore:696
[Getting data:]######################################################cashflow_data(2017-1) total:696, success:4, ignore:692
[Getting data:]#Fail during getting report data on 2017-2, exception:获取失败，请检查网络和URL
[Getting data:]#Fail during getting profit data on 2017-2, exception:获取失败，请检查网络和URL
[Getting data:]#Fail during getting operation data on 2017-2, exception:获取失败，请检查网络和URL


In [14]:
ListItemExists('陆家嘴',2004,1,'report_data')

True

### Retrieve Stock Transaction Data

#### 历史行情
>```py
import tushare as ts
ts.get_hist_data('600848') #一次性获取全部日k线数据
```

本接口只能获取近3年的日线数据，适合搭配均线数据进行选股和分析，如果需要全部历史数据，请调用下一个接口get_h_data()。


参数说明：

* code：股票代码，即6位数字代码，或者指数代码（sh=上证指数 sz=深圳成指 hs300=沪深300指数 sz50=上证50 zxb=中小板 cyb=创业板）
* start：开始日期，格式YYYY-MM-DD
* end：结束日期，格式YYYY-MM-DD
* ktype：数据类型，D=日k线 W=周 M=月 5=5分钟 15=15分钟 30=30分钟 60=60分钟，默认为D
* retry_count：当网络异常后重试次数，默认为3
* pause:重试时停顿秒数，默认为0

返回值说明:
* date：日期
* open：开盘价
* high：最高价
* close：收盘价
* low：最低价
* volume：成交量
* price_change：价格变动
* p_change：涨跌幅
* ma5：5日均价
* ma10：10日均价
* ma20:20日均价
* v_ma5:5日均量
* v_ma10:10日均量
* v_ma20:20日均量
* turnover:换手率[注：指数无此项]

#### 复权数据


>```py
df = ts.get_stock_basics()
date = df.ix['600848']['timeToMarket'] #上市日期YYYYMMDD
ts.get_h_data('002337') #前复权
ts.get_h_data('002337', autype='hfq') #后复权
ts.get_h_data('002337', autype=None) #不复权
ts.get_h_data('002337', start='2015-01-01', end='2015-03-16') #两个日期之间的前复权数据
ts.get_h_data('399106', index=True) #深圳综合指数
```

参数说明：

* code:string,股票代码 e.g. 600848
* start:string,开始日期 format：YYYY-MM-DD 为空时取当前日期
* end:string,结束日期 format：YYYY-MM-DD 为空时取去年今日
* autype:string,复权类型，qfq-前复权 hfq-后复权 None-不复权，默认为qfq
* index:Boolean，是否是大盘指数，默认为False
* retry_count : int, 默认3,如遇网络等问题重复执行的次数
* pause : int, 默认 0,重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题

返回值说明：

* date : 交易日期 (index)
* open : 开盘价
* high : 最高价
* close : 收盘价
* low : 最低价
* volume : 成交量
* amount : 成交金额

In [19]:
def BuildStockHistoryData(df, doc):
    # doc['history_data']=[]
    index_name=df.index.name
    for i in df.index:
        item={}
        item[index_name]=str(i.date())
        for v in df.ix[str(i.date())]:        
            item[v]=df.ix[str(i.date())][v].values[0]
        doc['history_data'].append(item)   

In [199]:
def CleanStockHistoryData():
    for i in c_stock_list.find():
        i['history_data']=[]
        c_stock_list.replace_one({'stock_id':i['stock_id']},i)

In [234]:
def check_duplicated_history_data(stock_id='',date=''):
    s=c_stock_list.find({'stock_id':stock_id})[0]
    h=s['history_data']
    df=DataFrame.from_dict(h)
    return df 

def convert_history_date_type():
    s=c_stock_list.find()
    for i in s:
        process=0
        total=0
        h=i['history_data']
        for item in h:
            total+=1
            if type(item['date']) is str:
                # Need to convert
                item['date']=datetime.strptime(item['date'],'%Y-%m-%d')
                process+=1
        c_stock_list.replace_one({'stock_id':i['stock_id']},i)
        print('Converted %s/%s for %s %s' %(process,total,i['stock_id'],i['stock_name']))
        
def get_stock_history_data(stock_id):
    s=c_stock_list.find({'stock_id':stock_id})[0]
    return s['history_data']


# #1 Experiment: Duplication detect method comparison
Method 1

```py
if lst.count(i)>1:
  list.remove(i)
```

Method 2
Convert it to Dataframe

```py
```

In [279]:
l=get_stock_history_data('600298')
len_original=len(l)
l_new=l.copy()
for i in l_new:
    if l_new.count(i)>1:
        l_new.remove(i)

len_list_new=len(l_new)

df=DataFrame.from_dict(l)
len_df_original=df.count()
df_drop=df.drop_duplicates()
len_df_new=df_drop.count()

print('Original list length:%s, length removed duplicates:%s, \nDataframe length original:%s, length removed duplicates:%s' %(len_original, 
                                                                                                                            len_list_new, 
                                                                                                                            len_df_original,
                                                                                                                           len_df_new))

Original list length:4041, length removed duplicates:4016, 
Dataframe length original:amount    4041
close     4041
date      4041
high      4041
low       4041
open      4041
volume    4041
dtype: int64, length removed duplicates:amount    3997
close     3997
date      3997
high      3997
low       3997
open      3997
volume    3997
dtype: int64


In [297]:
#df['date']=df.to_datetime(df['date'],format='%Y-%m-%d')
l_new[0]['date'] in df.sort_values('date')['date']

TypeError: argument of type 'Timestamp' is not iterable

In [286]:

for x in l_new:
    if x['date'] in df['date']:
        print('%s found.' %x['date'].date())
        l_new.remove(x)
    else:
        print('%s not found.' %x['date'])

print(DataFrame.from_dict(l_new).sort('date'))

2017-06-02 00:00:00 not found.
2017-06-01 00:00:00 not found.
2017-05-31 00:00:00 not found.
2017-05-26 00:00:00 not found.
2017-05-25 00:00:00 not found.
2017-05-24 00:00:00 not found.
2017-05-23 00:00:00 not found.
2017-05-22 00:00:00 not found.
2017-05-19 00:00:00 not found.
2017-05-18 00:00:00 not found.
2017-05-17 00:00:00 not found.
2017-05-16 00:00:00 not found.
2017-05-15 00:00:00 not found.
2017-05-12 00:00:00 not found.
2017-05-11 00:00:00 not found.
2017-05-10 00:00:00 not found.
2017-05-09 00:00:00 not found.
2017-05-08 00:00:00 not found.
2017-05-05 00:00:00 not found.
2017-05-04 00:00:00 not found.
2017-05-03 00:00:00 not found.
2017-05-02 00:00:00 not found.
2017-04-28 00:00:00 not found.
2017-04-27 00:00:00 not found.
2017-04-26 00:00:00 not found.
2017-04-25 00:00:00 not found.
2017-04-24 00:00:00 not found.
2017-04-21 00:00:00 not found.
2017-04-20 00:00:00 not found.
2017-04-19 00:00:00 not found.
2017-04-18 00:00:00 not found.
2017-04-17 00:00:00 not found.
2017-04-

C:\Users\wellszha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [263]:
df_drop.sort('date')

C:\Users\wellszha\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,amount,close,date,high,low,open,volume
3991,675590800.0,3.97,2000-08-18,4.08,3.67,3.67,22387200.0
3990,171316348.0,3.90,2000-08-21,4.01,3.88,3.93,5588600.0
3989,77402892.0,3.79,2000-08-22,3.91,3.79,3.90,2592400.0
3988,92110514.0,3.72,2000-08-23,3.85,3.63,3.76,3184900.0
3987,47679265.0,3.79,2000-08-24,3.82,3.69,3.73,1631400.0
3986,49144373.0,3.82,2000-08-25,3.91,3.75,3.79,1642300.0
3985,38746585.0,3.74,2000-08-28,3.81,3.71,3.79,1329900.0
3984,27365538.0,3.74,2000-08-29,3.76,3.71,3.72,944400.0
3983,22539642.0,3.76,2000-08-30,3.85,3.74,3.74,762200.0
3982,25894555.0,3.67,2000-08-31,3.75,3.66,3.74,898300.0


In [224]:
#json.loads(df_drop.T.to_json()).values()
lst=list(json.loads(df_drop.T.to_json()).values())

In [226]:
lst[0]

{'amount': 100110944.0,
 'close': 6.37,
 'date': 1380240000000,
 'high': 6.67,
 'low': 6.27,
 'open': 6.27,
 'volume': 6124597.0}

In [70]:
df_test=ts.get_h_data('002168','2017-01-20','2017-06-13')

[Getting data:]#

In [73]:
df_test

,open,high,close,low,volume,amount
date,,,,,,
2017-01-20,16.41,17.0,16.99,16.41,16045161.0,268691334.0


In [124]:
df_test.ix['2017-01-20']['open']

16.41

In [127]:
index_name=df_test.index.name
for i in df_test.index:
    a[index_name]=i
    for j in df_test.columns.values:
        a[j]=df_test.ix[i][j]
    
    print(a)

{'amount': 268691334.0, 'volume': 16045161.0, 'low': 16.41, 'close': 16.989999999999998, 'open': 16.41, 'high': 17.0, 'date': Timestamp('2017-01-20 00:00:00')}


In [115]:
df_test.columns.values

array(['open', 'high', 'close', 'low', 'volume', 'amount'], dtype=object)

In [62]:
def print_array_order(array=None, array_to_sort='',action=''):
    print('============= %s' %action)
    for i in array:
        print(len(i[array_to_sort]))
        
def sort_document_array(array=None, array_to_sort='', acsend=True):
    if array is None:
        return None
    else:
        result=[]
        l=0
        for i in array:
            l=len(i[array_to_sort])
            offset=0
            quit=False
            
            if len(result)==0:
                # Emplty list, just append
                result.append(i)
                #print_array_order(result,array_to_sort,'append at empty')
            else:
                
                for ii in result:
                    len_of_result=len(ii[array_to_sort])
                    # print('comparing array item(%s) > result array(%s):%s' %(l,len_of_result,l>len_of_result))
                    if acsend:
                        if len_of_result>l and not quit:
                            result.insert(offset,i)
                            quit=True
                            #print_array_order(result,array_to_sort,'insert')
                        offset+=1
                        
                if not quit:
                    # Still no quit means no small array found, just append
                    result.append(i)
                    #print_array_order(result,array_to_sort,'append at last')
                    
        return result

In [64]:
x=sort_document_array(c_stock_list.find(),'history_data')
print_array_order(x,'history_data','append at last')

============= append at last
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [74]:
i=c_stock_list.find({'stock_id':'002168'})

In [76]:
i[0]

{'_id': ObjectId('58c94466ed2c880e4c8909b3'),
 'area': '深圳',
 'bvps': 1.64,
 'cashflow_data': [{'cashflowratio': nan,
   'cf_liabilities': nan,
   'cf_nm': 0.4044,
   'cf_sales': 0.0922,
   'quarter': 3,
   'rateofreturn': nan,
   'year': 2006},
  {'cashflowratio': -3.0667,
   'cf_liabilities': -0.0274,
   'cf_nm': -0.0863,
   'cf_sales': -0.0181,
   'quarter': 3,
   'rateofreturn': -0.004,
   'year': 2007},
  {'cashflowratio': 0.6222,
   'cf_liabilities': 0.0061,
   'cf_nm': nan,
   'cf_sales': 0.079,
   'quarter': 1,
   'rateofreturn': 0.002,
   'year': 2017},
  {'cashflowratio': 5.5912,
   'cf_liabilities': 0.0551,
   'cf_nm': nan,
   'cf_sales': 0.3269,
   'quarter': 1,
   'rateofreturn': 0.0093,
   'year': 2016},
  {'cashflowratio': 2.4714,
   'cf_liabilities': 0.0245,
   'cf_nm': nan,
   'cf_sales': 0.0542,
   'quarter': 2,
   'rateofreturn': 0.0042,
   'year': 2016},
  {'cashflowratio': 2.4208,
   'cf_liabilities': 0.0235,
   'cf_nm': 2.5981,
   'cf_sales': 0.0717,
   'quarter':

In [64]:
d

,open,high,close,low,volume,amount
date,,,,,,
2017-06-12,24.77,24.77,24.32,24.06,10008397.0,244115671.0
2017-06-09,24.28,24.85,24.72,24.24,8175023.0,200946673.0
2017-06-08,24.15,24.80,24.32,24.09,11660488.0,285394951.0
2017-06-07,23.95,24.30,24.28,23.76,12004666.0,288838013.0
2017-06-06,22.83,23.93,23.89,22.66,12879959.0,302000865.0
2017-06-05,22.53,22.99,22.81,22.47,6944436.0,158231778.0


In [68]:
processed=0
total=c_stock_list.count()

for s in sort_document_array(c_stock_list.find(no_cursor_timeout=True),'history_data'):
    stock_id=s['stock_id']
    stock_name=s['stock_name']
    if 'timeToMarket' not in s:
        #print('Cant find timeToMarket in %s' %stock_name)
        pass
    elif s['timeToMarket']==0:
        #print('Mistake timeToMarket value(%s) in %s.' %(s['timeToMarket'],stock_name))
        pass
    else:
        start=''
        if 'history_data' not in s or len(s['history_data'])<1:
            # 还没有history_data，以timeToMarket作为获取历史交易数据的起点
            time_to_market=str(s['timeToMarket'])
            start=time_to_market[0:4]+'-'+time_to_market[4:6]+'-'+time_to_market[6:8]
        else:
            # 已经有history_data，寻找最近的历史交易日期，将其作为历史交易数据的起点
            # 由于当前存在错误，所以采用更新操作，以后会采用增加操作
            start=s['history_data'][0]['date']
        end=str(datetime.now().date())
        print('\nRetrieve history data for stock:%s %s(%s to %s)' %(stock_id,stock_name,start,end))
        try:
            df_history=ts.get_h_data(stock_id,start,end)
        
            BuildStockHistoryData(df_history,s)
            c_stock_list.replace_one({'stock_id':stock_id},s)
            processed+=1
        except Exception as e:
            print('Error occured during process, error message:%s' %e)
            
        print('\n%s records added, overall %s/%s processed.' %(len(df_history),processed,total))
        


Retrieve history data for stock:300121 阳谷华泰(2010-09-17 to 2017-06-06)
[Getting data:]###########################
1589 records added, overall 1/752 processed.

Retrieve history data for stock:002352 顺丰控股(2010-02-05 to 2017-06-06)
[Getting data:]#############################
1723 records added, overall 2/752 processed.

Retrieve history data for stock:002386 天原集团(2010-04-09 to 2017-06-06)
[Getting data:]############################
1692 records added, overall 3/752 processed.

Retrieve history data for stock:600577 精达股份(2002-09-11 to 2017-06-06)
[Getting data:]###########################################################
3289 records added, overall 4/752 processed.

Retrieve history data for stock:300398 飞凯材料(2014-10-09 to 2017-06-06)
[Getting data:]##########
559 records added, overall 5/752 processed.

Retrieve history data for stock:002372 伟星新材(2010-03-18 to 2017-06-06)
[Getting data:]#############################
1741 records added, overall 6/752 processed.

Retrieve history data for 

## Stock fields transformation

### Generate stock id alias
To generate stock id alias from existing stock_id and modify stock_id to pure stock id without region code as prefix.

Target collection: StockList

Exception control(Check before dong the transformation)
* stock_id has sh/sz as prefix
* stock_id_alias is not exist

In [148]:
records = c_stock_list.find()
for r in records:
    for region in ['sh','sz','k_hk']:
        if region in r['stock_id']:
            r['stock_id_alias'] = r['stock_id']
            r['stock_id'] = r['stock_id'].lstrip(region)
            r['market_agency'] = region
            print(r)
            c_stock_list.replace_one({'stock_id':r['stock_id_alias']},r)

{'related_articles': [{'url': 'http://finance.sina.com.cn/roll/2017-03-14/doc-ifychihc6447842.shtml', 'article_name': '港股通资金南下扫货 中资地产股或迎价值重估'}], 'market_agency': 'k_hk', 'stock_id': 'CSCSHQ', '_id': ObjectId('58ca056eed2c88738049f6d4'), 'stock_id_alias': 'k_hkCSCSHQ', 'stock_name': '沪港通'}
{'related_articles': [{'url': 'http://finance.sina.com.cn/roll/2017-03-14/doc-ifychihc6447842.shtml', 'article_name': '港股通资金南下扫货 中资地产股或迎价值重估'}], 'market_agency': 'k_hk', 'stock_id': '01918', '_id': ObjectId('58ca056eed2c88738049f6d6'), 'stock_id_alias': 'k_hk01918', 'stock_name': '融创中国'}
{'related_articles': [{'url': 'http://finance.sina.com.cn/roll/2017-03-14/doc-ifychihc6447842.shtml', 'article_name': '港股通资金南下扫货 中资地产股或迎价值重估'}], 'market_agency': 'k_hk', 'stock_id': '01030', '_id': ObjectId('58ca056eed2c88738049f6d8'), 'stock_id_alias': 'k_hk01030', 'stock_name': '新城发展控股'}
{'related_articles': [{'url': 'http://finance.sina.com.cn/roll/2017-03-14/doc-ifychihc6447842.shtml', 'article_name': '港股通资金南下扫货 中

### 更新Stock List加上Realted articles的_id

In [63]:
total_stock = c_stock_list.count()
current_stock = 0
for stock in c_stock_list.find():
    current_stock += 1    
    stock_articles = stock['related_articles']
    print('Updating %s/%s article id for %s' %(current_stock,
                                               total_stock,
                                               stock['stock_name']), end='...')
    _total = len(stock_articles)
    _updated = 0
    _passed = 0
    for a in stock_articles:
        # stock[0]['url']
        if not 'article_id' in a.keys():
            article = c_article_body.find_one({'url':a['url']})
            article_id = article['_id']
            a['article_id']=article['_id']
            #print('article id for %s updated.' %a['article_name'])
            _updated += 1
        else:
            #print('article id for %s exists, pass.' %a['article_name'])
            _passed += 1
    c_stock_list.replace_one({'_id':stock['_id']},stock)

    print('Completed. %s/%s updated, %s passed' %(_updated,
                                                _total,
                                                _passed))


Updating 1/3224 article id for 方正证券...Completed. 4/253 updated, 249 passed
Updating 2/3224 article id for 沪深300...Completed. 23/703 updated, 680 passed
Updating 3/3224 article id for 国信证券...Completed. 9/276 updated, 267 passed
Updating 4/3224 article id for 云海金属...Completed. 0/20 updated, 20 passed
Updating 5/3224 article id for 长城电脑...Completed. 0/15 updated, 15 passed
Updating 6/3224 article id for 浙报传媒...Completed. 2/28 updated, 26 passed
Updating 7/3224 article id for 雏鹰农牧...Completed. 3/37 updated, 34 passed
Updating 8/3224 article id for 兴森科技...Completed. 0/13 updated, 13 passed
Updating 9/3224 article id for 中国联通...Completed. 0/161 updated, 161 passed
Updating 10/3224 article id for 顺丰控股...Completed. 2/61 updated, 59 passed
Updating 11/3224 article id for 国海证券...Completed. 1/160 updated, 159 passed
Updating 12/3224 article id for 格林美...Completed. 0/31 updated, 31 passed
Updating 13/3224 article id for 东方航空...Completed. 1/67 updated, 66 passed
Updating 14/3224 article id for 雅百特.

## 更新Stock List加上Realted articles的Date

In [4]:
# Function to retrieve the article date from input url
def get_article_date_from_url(url):
    pattern = re.compile(r'[0-9]{4,4}-?[0-9]{2,2}-?[0-9]{2,2}/')

    article_url = url
    article_date = pattern.search(article_url)
    
    if article_date is None:
        print('No matched article date from url %s.' %article_url)
    else:
        a_date=article_date.group().replace('/','')
        if len(a_date) != 10:
            a_date = '%s-%s-%s' %(a_date[0:4],a_date[4:6],a_date[6:8])

        return a_date
            

In [62]:
total_stock = c_stock_list.count()
current_stock = 0
for stock in c_stock_list.find():
    current_stock += 1    
    stock_articles = stock['related_articles']
    print('Updating %s/%s article date for %s' %(current_stock,
                                               total_stock,
                                               stock['stock_name']), end='...')
    _total = len(stock_articles)
    _updated = 0
    _passed = 0
    for a in stock_articles:
        # stock[0]['url']
        if not 'article_date' in a.keys():
            a['article_date']=get_article_date_from_url(a['url'])
            # print('fetched date(%s) from url.' %a['article_date'])
            _updated += 1
        else:
            #print('article id for %s exists, pass.' %a['article_name'])
            _passed += 1
            
    # disable database operation for test model
    c_stock_list.replace_one({'_id':stock['_id']},stock)

    print('Completed. %s/%s updated, %s passed' %(_updated,
                                                _total,
                                                _passed))


Updating 1/3224 article date for 方正证券...Completed. 253/253 updated, 0 passed
Updating 2/3224 article date for 沪深300...Completed. 703/703 updated, 0 passed
Updating 3/3224 article date for 国信证券...Completed. 276/276 updated, 0 passed
Updating 4/3224 article date for 云海金属...Completed. 20/20 updated, 0 passed
Updating 5/3224 article date for 长城电脑...Completed. 15/15 updated, 0 passed
Updating 6/3224 article date for 浙报传媒...Completed. 28/28 updated, 0 passed
Updating 7/3224 article date for 雏鹰农牧...Completed. 37/37 updated, 0 passed
Updating 8/3224 article date for 兴森科技...Completed. 13/13 updated, 0 passed
Updating 9/3224 article date for 中国联通...Completed. 161/161 updated, 0 passed
Updating 10/3224 article date for 顺丰控股...Completed. 61/61 updated, 0 passed
Updating 11/3224 article date for 国海证券...Completed. 160/160 updated, 0 passed
Updating 12/3224 article date for 格林美...Completed. 31/31 updated, 0 passed
Updating 13/3224 article date for 东方航空...Completed. 67/67 updated, 0 passed
Updating 14

In [64]:
c_article_body.find()[0]

{'_id': ObjectId('58b0549ded2c88418c9b9085'),
 'article_body': '新浪财经App机会将至一直以来，内地股票和港股之间，都存在多多少少的价差。那么面对这种情况，尤其在当前香港市场受到的关注程度无比之高的时候，我们应该如何理解这个价差？两地股市的价值除了传统的企业价值以外，还有多少差异？这里，就让我们从各个不同的点，来讨论这个问题。1、内地股市并不是永远比港股贵在过去几年，绝大多数股票在绝大多数时候，A股的估值都比港股要贵。但是，这种情况并不出现在100%的情况里。一个例子出现在极端的A股大熊市，即2005年的夏天。彼时，港股的估值往往比内地股票更贵。当然，之后A股走出了惊人的大牛市行情。另一个情况则出现在一些个股身上，有时候一些公司的股票，在香港的估值比内地更高。这种倒挂有时候是由于内地股票盘子太大、因此估值变得太便宜，有时候由于海外资本太喜欢这个公司的股票、使得香港估值更贵，有时候则两种因素兼而有之。两地估值差的产生具有随机性，而不完全由必然性决定：尽管大部分分析试图找出决定两地价差的必然决定因素，试图做到一劳永逸的解决这个问题的分析路径，但是实际上，我们需要承认，这个价差存在太多的决定因素，因此非常难以被彻底定性，随机性在很大程度上决定了这个价差究竟有多大。也就是说，当一个市场的股价已经包含了大量随机性的时候，一个由两地股价决定的价差，其随机性必然大增。如果忽视了这一点，我们就很难理解这种价差的产生。2、股息分红税率不同导致同股不同价值目前，香港的股息分红税率比较高，在20%左右，而内地的分红税率则为持有1个月以内20%、1个月到1年为10%、1年以上为5%。不同的税率导致在价格相同的情况下，内地股票必然比香港的更有价值。反之，这也就意味着在价值相等的时候，内地股票应该比香港更贵。3、打新效应导致内地股票价值更高比分红税率赋予了内地股票更多价值的，是其秉承的申购新股功能。在目前的新股申购政策下，只有持有市值才能申购新股，而新股上市则几乎不可能破发。对于机构投资者来说，这意味着每年几个百分点的无风险收益加成，而对资金量更小的个人投资者来说，这意味着每年20%左右的无风险收益加成，尽管这种加成有一定的随机性。而在香港，这种效应则完全不存在。因此，内地市场在估值同样的情况下，必然比香港股票更有价值。不过，对于

In [56]:
c_stock_list.replace_one({'_id':stock['_id']},stock).matched_count

1

In [8]:
for i in db.StockList.find()[0]['related_articles']:
    id = db.ArticleBody.find_one({'url':i['url']})
    print('%s - %s(%s)' %(id['_id',i['article_name'],i['url']))

{'url': 'http://finance.sina.com.cn/stock/marketresearch/2017-02-24/doc-ifyavvsh6483169.shtml', 'article_body': '新浪财经App机会将至自春节以来，港股、A股的部分板块便呈现出牛市行情，沪指、深成指也在节后处于震荡上行的状态。一个普遍关心的问题是，这种行情能持续多久？近日方正证券首席经济学家、分析师“网红”任泽平更是发表了以《港股已牛市，A股结构性牛市还会远吗？》为题的文章，预测今年A股将走出结构性牛市的行情。难道今年A股慢牛真的来了？节后A股呈现上行趋势从大盘来看，节后A股呈现出明显的上行趋势。截至2月24日，上证、深证、创业板、沪深300的市盈率分别为16.69、45.93、60.47、13.62倍，市场整体估值水平大幅改善。国信证券融资融券部近日发布文章认为，今年随着新股加速，市场局部泡沫可能会继续下降，但蓝筹股、绩优股、周期股等因估值优势突出，受到的影响则相对较小，成为阶段性行情的主要脉络。结合去年融资压力大、市场资金偏紧的问题，年初监管层从股指期货、再融资两个角度为市场资金开源节流，加上市场估值水平不算太高，市场结构性机会比去年增多。长城证券分析师汪毅近期在研报中表示，近期地方国企改革政策持续推出，养老金开始委托进行投资运营，股指期货松绑，再融资新规推出，这些都能提升市场情绪，因此维持对春季行情的乐观看法，且维持对春季行情中蓝筹占优的判断。盘面上看，对比春节以来的2月3日的首个交易日，深证成指上涨4.39%，上证综指上涨3.6%。Wind数据统计显示，2月24日沪深两市所有交易个股涨跌比为1781：1017，市场总体赚钱效应为63%，市场偏强。具体到本周，沪指先扬后抑，上半周沪指缩量冲高，一度冲高至3264点，上方阻力较大，且成交量缩小缺乏上攻动力，下半周沪指逐渐回落，但调整幅度较小，周五尾盘依然报收红盘。截至2月24日，从行业来看，申万一级行业所有行业的周指数全部上涨，计算机、传媒、通信等行业的周指数涨幅超过3%。概念股方面，次新股、水泥股、有色金属等均有所表现，龙头个股表现突出。资金流向方面，2月24日主力资金净流入金额为-175.22亿元。其中开盘主力资金净流入为-119.16亿元，尾盘主力资金净流入3.38亿元。无论是市场还是从各券商的分

# Convert StockList timeToMarket to Datetime type

In [23]:
for i in c_stock_list.find():
    #print('processing %s %s...' %(i['stock_id'], i['stock_name']), end='')
    if 'timeToMarket' in i:
        #print('found timeToMarket, type is %s... ' %type(i['timeToMarket']), end='')
        if type(i['timeToMarket']) is not datetime:            
            if str(i['timeToMarket'])=='0':
                print('found mistake datetime value %s, getting data from tushare...' %i['timeToMarket'], end='')
                i['timeToMarket']=get_stock_timeToMarket(i['stock_id'])
                print('retrieved success, timeToMarket=%s' %i['timeToMarket'])
            i['timeToMarket']=datetime.strptime(str(i['timeToMarket']), '%Y%m%d')
            c_stock_list.replace_one({'stock_id':i['stock_id']},i)
        else:
            #print('Already the datetime type, bypass.')
            pass
    else:
        print('%s(%s) no timeToMarket found. getting data from tushare...' %(i['stock_name'],i['stock_id']), end='')
        i['timeToMarket']=get_stock_timeToMarket(i['stock_id'])
        print('retrieved success, timeToMarket=%s' %i['timeToMarket'])
        i['timeToMarket']=datetime.strptime(str(i['timeToMarket']), '%Y%m%d')
        c_stock_list.replace_one({'stock_id':i['stock_id']},i)

沪深300指数(000300) no timeToMarket found. getting data from tushare...

Exception: No timeToMarket value found for 000300